In [1]:
import os
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage
from IPython.display import display, Markdown, Latex
from datetime import datetime

In [2]:
API_KEY = os.environ['MISTRAL_API_KEY']

MODEL_ID = 'open-mixtral-8x22b'
MODEL_SEED = 3
MODEL_TEMPERATURE = 0.7

file_system_prompt_1 = open("../../system_prompt_1.txt", "r")
file_system_prompt_2 = open("../../system_prompt_2.txt", "r")
SYSTEM_PROMPT_1 = file_system_prompt_1.read()
SYSTEM_PROMPT_2 = file_system_prompt_2.read()
file_system_prompt_1.close()
file_system_prompt_2.close()

FILE_PATH_PROBLEM = '../../../../Datasets/IP_1_Radiotherapy-Treatment/'
assert(FILE_PATH_PROBLEM != '../../../../Datasets/')
file_problem_description = open(FILE_PATH_PROBLEM + 'ProblemDescription.txt', 'r')
PROBLEM_DESCRIPTION = file_problem_description.read()
file_problem_description.close()


client = MistralClient(api_key=API_KEY)

print(f'Time of execution: {datetime.now()}')

Time of execution: 2024-05-17 16:36:21.573340


## Step 1 - Generate Mathematical Formulation 

In [3]:
messages_1 = [
    ChatMessage(role="system", content=SYSTEM_PROMPT_1),
    ChatMessage(role="user", content=PROBLEM_DESCRIPTION)
]

In [4]:
response_1 = client.chat(
    model=MODEL_ID,
    messages=messages_1,
    random_seed=MODEL_SEED,
    temperature=MODEL_TEMPERATURE 
)

response_1_text = response_1.choices[0].message.content

In [5]:
Markdown(response_1_text)

Parameters:
- \( n \) is the number of patients, \( n = 17 \)
- \( C \) is the total maximal capacity of proton fractions, \( C = 100 \)
- \( BED_{i}(p, 15-p) \) is the Biological Equivalent Dose score for patient \( i \) when using \( p \) proton fractions and \( 15-p \) photon fractions.

Decision Variables:
- \( x_{ip} \in \{0, 1\} \) is a binary decision variable that equals 1 if patient \( i \) receives \( p \) proton fractions and 0 otherwise.

Objective Function:
Maximize the total BED scores for all patients:
\[ \text{maximize} \quad \sum_{i=1}^{n} \sum_{p=0}^{15} BED_{i}(p, 15-p) \cdot x_{ip} \]

Constraints:
1. Each patient should receive exactly 15 fractions:
\[ \sum_{p=0}^{15} x_{ip} = 1 \quad \text{for} \quad i = 1, 2, \dots, n \]

2. The total number of proton fractions does not exceed the maximum capacity:
\[ \sum_{i=1}^{n} \sum_{p=0}^{15} p \cdot x_{ip} \leq C \]

In summary, the integer linear optimization model to solve the problem is:

\[
\begin{align*}
\text{maximize} \quad & \sum_{i=1}^{n} \sum_{p=0}^{15} BED_{i}(p, 15-p) \cdot x_{ip} \\
\text{subject to} \quad & \sum_{p=0}^{15} x_{ip} = 1 \quad \text{for} \quad i = 1, 2, \dots, n \\
& \sum_{i=1}^{n} \sum_{p=0}^{15} p \cdot x_{ip} \leq C \\
& x_{ip} \in \{0, 1\}
\end{align*}
\]

In [6]:
print(response_1_text)

Parameters:
- \( n \) is the number of patients, \( n = 17 \)
- \( C \) is the total maximal capacity of proton fractions, \( C = 100 \)
- \( BED_{i}(p, 15-p) \) is the Biological Equivalent Dose score for patient \( i \) when using \( p \) proton fractions and \( 15-p \) photon fractions.

Decision Variables:
- \( x_{ip} \in \{0, 1\} \) is a binary decision variable that equals 1 if patient \( i \) receives \( p \) proton fractions and 0 otherwise.

Objective Function:
Maximize the total BED scores for all patients:
\[ \text{maximize} \quad \sum_{i=1}^{n} \sum_{p=0}^{15} BED_{i}(p, 15-p) \cdot x_{ip} \]

Constraints:
1. Each patient should receive exactly 15 fractions:
\[ \sum_{p=0}^{15} x_{ip} = 1 \quad \text{for} \quad i = 1, 2, \dots, n \]

2. The total number of proton fractions does not exceed the maximum capacity:
\[ \sum_{i=1}^{n} \sum_{p=0}^{15} p \cdot x_{ip} \leq C \]

In summary, the integer linear optimization model to solve the problem is:

\[
\begin{align*}
\text{maximiz

## Step 2 - Generate the Pyomo Code

In [7]:
messages_2 = [
    ChatMessage(role="system", content=SYSTEM_PROMPT_2),
    ChatMessage(role="user", content=response_1_text)
]

In [8]:
response_2 = client.chat(
    model=MODEL_ID,
    messages=messages_2,
    random_seed=MODEL_SEED,
    temperature=MODEL_TEMPERATURE
)

response_2_text = response_2.choices[0].message.content

In [9]:
Markdown(response_2_text)

To solve the given optimization problem using Pyomo, you can use the following code. Note that I will use a sample data for the BED scores, as it was not provided in the problem description.

```python
import pyomo.environ as pyo

# Sample data for BED scores (replace with actual data)
n = 17
C = 100
BED = {i: [(p, (i+1)*(p+1)) for p in range(16)] for i in range(1, n+1)}

# Create the Pyomo model
model = pyo.ConcreteModel()

# Define the decision variables
model.x = pyo.Var(range(1, n+1), range(16), domain=pyo.Binary)

# Define the objective function
def obj_rule(model):
    return sum(BED[i][p-1][1] * model.x[i, p-1] for i in range(1, n+1) for p in range(1, 16))
model.obj = pyo.Objective(rule=obj_rule, sense=pyo.maximize)

# Define the constraints
def constraint1_rule(model, i):
    return sum(model.x[i, p-1] for p in range(1, 16)) == 1
model.constraint1 = pyo.Constraint(range(1, n+1), rule=constraint1_rule)

def constraint2_rule(model):
    return sum(p * model.x[i, p-1] for i in range(1, n+1) for p in range(1, 16)) <= C
model.constraint2 = pyo.Constraint(rule=constraint2_rule)

# Solve the model
solver = pyo.SolverFactory('glpk')
solver.solve(model)

# Print the results
for i in range(1, n+1):
    for p in range(1, 16):
        if pyo.value(model.x[i, p-1]) > 0.5:
            print(f"Patient {i} receives {p} proton fractions.")
print(f"Total BED score: {pyo.value(model.obj)}")
```

This code creates a Pyomo model with the given objective function and constraints, and then solves the model using the GLPK solver. Replace the sample BED scores with the actual data to get the correct results.

In [10]:
print(response_2_text)

To solve the given optimization problem using Pyomo, you can use the following code. Note that I will use a sample data for the BED scores, as it was not provided in the problem description.

```python
import pyomo.environ as pyo

# Sample data for BED scores (replace with actual data)
n = 17
C = 100
BED = {i: [(p, (i+1)*(p+1)) for p in range(16)] for i in range(1, n+1)}

# Create the Pyomo model
model = pyo.ConcreteModel()

# Define the decision variables
model.x = pyo.Var(range(1, n+1), range(16), domain=pyo.Binary)

# Define the objective function
def obj_rule(model):
    return sum(BED[i][p-1][1] * model.x[i, p-1] for i in range(1, n+1) for p in range(1, 16))
model.obj = pyo.Objective(rule=obj_rule, sense=pyo.maximize)

# Define the constraints
def constraint1_rule(model, i):
    return sum(model.x[i, p-1] for p in range(1, 16)) == 1
model.constraint1 = pyo.Constraint(range(1, n+1), rule=constraint1_rule)

def constraint2_rule(model):
    return sum(p * model.x[i, p-1] for i in ran

### Code Executability

In [11]:
import pyomo.environ as pyo

# Sample data for BED scores (replace with actual data)
n = 17
C = 100
BED = {i: [(p, (i+1)*(p+1)) for p in range(16)] for i in range(1, n+1)}

# Create the Pyomo model
model = pyo.ConcreteModel()

# Define the decision variables
model.x = pyo.Var(range(1, n+1), range(16), domain=pyo.Binary)

# Define the objective function
def obj_rule(model):
    return sum(BED[i][p-1][1] * model.x[i, p-1] for i in range(1, n+1) for p in range(1, 16))
model.obj = pyo.Objective(rule=obj_rule, sense=pyo.maximize)

# Define the constraints
def constraint1_rule(model, i):
    return sum(model.x[i, p-1] for p in range(1, 16)) == 1
model.constraint1 = pyo.Constraint(range(1, n+1), rule=constraint1_rule)

def constraint2_rule(model):
    return sum(p * model.x[i, p-1] for i in range(1, n+1) for p in range(1, 16)) <= C
model.constraint2 = pyo.Constraint(rule=constraint2_rule)

# Solve the model
solver = pyo.SolverFactory('glpk')
solver.solve(model)

# Print the results
for i in range(1, n+1):
    for p in range(1, 16):
        if pyo.value(model.x[i, p-1]) > 0.5:
            print(f"Patient {i} receives {p} proton fractions.")
print(f"Total BED score: {pyo.value(model.obj)}")

Patient 1 receives 1 proton fractions.
Patient 2 receives 1 proton fractions.
Patient 3 receives 1 proton fractions.
Patient 4 receives 1 proton fractions.
Patient 5 receives 1 proton fractions.
Patient 6 receives 1 proton fractions.
Patient 7 receives 1 proton fractions.
Patient 8 receives 1 proton fractions.
Patient 9 receives 1 proton fractions.
Patient 10 receives 1 proton fractions.
Patient 11 receives 1 proton fractions.
Patient 12 receives 14 proton fractions.
Patient 13 receives 15 proton fractions.
Patient 14 receives 15 proton fractions.
Patient 15 receives 15 proton fractions.
Patient 16 receives 15 proton fractions.
Patient 17 receives 15 proton fractions.
Total BED score: 1459.0


### Solution Correctness

In [25]:
import pyomo.environ as pyo

# Sample data for BED scores (replace with actual data)
n = 17
C = 100
BED = {}
with open(FILE_PATH_PROBLEM + 'ProblemData.csv', 'r') as f:
    for i, line in enumerate(f):
        BED[i+1] = [float(x) for x in line.strip().split(',')]

# Create the Pyomo model
model = pyo.ConcreteModel()

# Define the decision variables
model.x = pyo.Var(range(1, n+1), range(16), domain=pyo.Binary)

# Define the objective function
def obj_rule(model):
    return sum(BED[i][p-1] * model.x[i, p-1] for i in range(1, n+1) for p in range(1, 16))
model.obj = pyo.Objective(rule=obj_rule, sense=pyo.maximize)

# Define the constraints
def constraint1_rule(model, i):
    return sum(model.x[i, p-1] for p in range(1, 16)) == 1
model.constraint1 = pyo.Constraint(range(1, n+1), rule=constraint1_rule)

def constraint2_rule(model):
    return sum(p * model.x[i, p-1] for i in range(1, n+1) for p in range(1, 16)) <= C
model.constraint2 = pyo.Constraint(rule=constraint2_rule)

# Solve the model
solver = pyo.SolverFactory('glpk')
solver.solve(model)

# Print the results
for i in range(1, n+1):
    for p in range(1, 16):
        if pyo.value(model.x[i, p-1]) > 0.5:
            print(f"Patient {i} receives {p} proton fractions.")
print(f"Total BED score: {pyo.value(model.obj)}")

Patient 1 receives 1 proton fractions.
Patient 2 receives 9 proton fractions.
Patient 3 receives 4 proton fractions.
Patient 5 receives 6 proton fractions.
Patient 6 receives 1 proton fractions.
Patient 7 receives 5 proton fractions.
Patient 8 receives 11 proton fractions.
Patient 9 receives 5 proton fractions.
Patient 10 receives 6 proton fractions.
Patient 11 receives 7 proton fractions.
Patient 12 receives 1 proton fractions.
Patient 13 receives 9 proton fractions.
Patient 14 receives 1 proton fractions.
Patient 15 receives 11 proton fractions.
Patient 16 receives 11 proton fractions.
Patient 17 receives 13 proton fractions.
Total BED score: 8.239999999999998
